<a href="https://colab.research.google.com/github/carloseducorinto/Agents/blob/main/L4_tools_customer_outreach_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [76]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
#!pip install -U langchain-cohere

In [77]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [78]:
from crewai import Agent, Task, Crew
import os
from google.colab import userdata
from langchain_cohere import ChatCohere

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [79]:
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

#### Cohere -  Initialize language model
#os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')
#llm = ChatCohere()



In [80]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CitiGroup, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [81]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of Citigroup's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [82]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [83]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [84]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [85]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [86]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [87]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [88]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [89]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [90]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [91]:
from IPython.display import Markdown
Markdown(result)

**Email 1: Subject: Empowering DeepLearningAI's Global Mission with Tailored Educational Solutions**

Dear Dr. Andrew Ng,

I hope this message finds you well. I'm writing to you not just as a representative of Citigroup, but as someone who deeply admires DeepLearningAI's commitment to democratizing AI education globally.

We've noticed your recent initiatives to expand course offerings on Coursera and understand the challenges such scaling efforts can entail. At Citigroup, we specialize in providing robust, scalable solutions tailored specifically to the needs of ed-tech platforms. Our collaboration could ensure that your platform remains resilient and efficient, even as demand continues to grow.

I would love the opportunity to discuss how we can support DeepLearningAI in achieving its mission. Are you available for a brief call next week?

Best regards,

[Your Name]

**Email 2: Subject: Collaborative Opportunities in AI Education**

Dear Dr. Andrew Ng,

Following our ongoing conversation about potential synergies between our organizations, I wanted to highlight another crucial area where Citigroup can add value—content creation and updates.

Staying ahead in AI and machine learning requires constant innovation and curriculum updates. Our team at Citigroup has extensive experience in integrating the latest AI breakthroughs into educational content. We can work together to ensure that your courses always reflect cutting-edge developments.

Could we explore this in more detail in our upcoming discussion?

Looking forward to your thoughts.

Warm regards,

[Your Name]

**Email 3: Subject: Localizing AI Education for Broader Impact**

Dear Dr. Andrew Ng,

As DeepLearningAI looks to enhance its global reach, addressing linguistic and regional educational nuances is essential. Citigroup has successfully helped other educational platforms develop localized content that resonates with a diverse audience.

Partnering with us could simplify the localization process for your courses, making AI education more accessible to non-English speakers worldwide.

I am eager to discuss how we can assist DeepLearningAI in this venture and would appreciate your insights on how best we can support your goals.

Please let me know a convenient time for you to talk next week.

Thankfully,

[Your Name]

These emails are tailored to address the specific needs identified in the profiling report, providing clear action steps and showcasing Citigroup's potential contributions to DeepLearningAI's success.